In [5]:
import os
import re
import requests
import csv
import shutil
import numpy as np
import pandas as pd
import bs4 as bs
import pandas_datareader as web
import itertools
from itertools import chain
import datetime as dt
from datetime import date
from datetime import timedelta
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

ModuleNotFoundError: No module named 'mlxtend'

In [4]:
pip install mlxtend

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'E:\\Softwares\\anaconda3\\Lib\\site-packages\\~klearn\\decomposition\\_cdnmf_fast.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2


In [ ]:
final_tickers=[]

def contains(s):
    return not re.match(r'[^.]+$', s)

def nifty50_tickers():
    link = 'https://en.wikipedia.org/wiki/NIFTY_50'
    response = requests.get(link)
    soup = bs.BeautifulSoup(response.text, 'lxml')
    table = soup.find('table', {'class' : 'wikitable sortable'}, {'id' : 'constituents'})
    tickers=[]
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[1].text[:]
        tickers.append(ticker)
    for i in tickers:
        if contains(i) != True:
            final_tickers.append(i)
            
    return final_tickers

nifty50_tickers()

In [ ]:
string=".NS"
new_tickerlist = [x + string for x in final_tickers]

In [ ]:
start = date.today() - timedelta(days=2190)
end = date.today()

if not os.path.exists('Companies'):
    os.makedirs('Companies')

for ticker in new_tickerlist:
    if not os.path.exists('Companies/{}.csv'.format(ticker)):
        print("{} - Fetching Data... ".format(ticker))
        df = web.DataReader(ticker, 'yahoo', start, end)
        df.to_csv('Companies/{}.csv'.format(ticker))
    else:
        print("{} - Already Fetched.!".format(ticker))

In [ ]:
main_df = pd.DataFrame()

for ticker in new_tickerlist:
    df = pd.read_csv('Companies/{}.csv'.format(ticker))
    df.set_index('Date', inplace=True)
    df.rename(columns = {'Close': ticker}, inplace=True)
    df.drop(columns = ['Open', 'High', 'Low', 'Adj Close', 'Volume'], inplace=True)
    if main_df.empty:
        main_df = df
    else:
        main_df = main_df.join(df, how='outer')
        
main_df = main_df.reset_index()
main_df.set_index('Date', inplace=True)
Dataframe = pd.concat([main_df.tail(1)])

In [ ]:
high = Dataframe.copy()
open1 = Dataframe.copy()
low = Dataframe.copy()
close = Dataframe.copy()

high.loc[:] = np.nan
open1.loc[:] = np.nan
low.loc[:] = np.nan
close.loc[:] = np.nan

Datelist=[]

if not os.path.exists("Companies_data"):
    os.makedirs("Companies_data")

for ticker in new_tickerlist:
    with open("Companies_data/{}.csv".format(ticker), 'w') as f:
        pass
    df = pd.read_csv('Companies/{}.csv'.format(ticker))
    data1 = pd.concat([df.tail(2)])
    data1.drop(df.tail(1).index,inplace=True)
    data1.to_csv("Companies_data\{}.csv".format(ticker), header=True, index=False)
    with open("Companies_data\{}.csv".format(ticker)) as f:
        companies_csv = csv.DictReader(f)
        for row in companies_csv:
            Date = row['Date']
            HighPrice = float(float(row['High']))
            OpenPrice = float(float(row['Open']))
            LowPrice = float(float(row['Low']))
            ClosePrice = float(float(row['Close']))
        H = HighPrice
        O = OpenPrice
        L = LowPrice
        C = ClosePrice
    Datelist.append(Date)
    high[ticker].fillna(H, inplace=True)
    open1[ticker].fillna(O, inplace=True)
    low[ticker].fillna(L, inplace=True)
    close[ticker].fillna(C, inplace=True)

In [ ]:
newdata = pd.DataFrame(columns=['Companies', '200_Moving_AVG', '50_Moving_AVG'])
newdata['Companies'] = new_tickerlist
mva50all, mva200all = ([] for i in range(2))

for tickers in new_tickerlist:
    df = pd.read_csv('Companies/{}.csv'.format(tickers))
    mva50 = round(df['Close'].rolling(window=50, min_periods=0).mean(), 3)
    mva200 = round(df['Close'].rolling(window=200, min_periods=0).mean(), 3)
    df['50_mVA'] = mva50
    df['200_mvA'] = mva200
    df.to_csv('Companies/{}.csv'.format(tickers), index=False)       
    mva50all.append(mva50.iloc[-1])
    mva200all.append(mva200.iloc[-1])
    
newdata['200_Moving_AVG'] = mva200all
newdata['50_Moving_AVG'] = mva50all

In [ ]:
high1,low1,open2,close1 = ([] for i in range(4))
    
for (colname, colval) in low.iteritems():
    low1.append(round(float(colval), 3))
    
for (colname, colval) in open1.iteritems():
    open2.append(round(float(colval), 3))

for (colname, colval) in high.iteritems():
    high1.append(round(float(colval), 3))
    
for (colname, colval) in close.iteritems():
    close1.append(round(float(colval), 3))

In [ ]:
predictive_values,confidence1,kfeature,kscore = ([] for i in range(4))

for ticker in new_tickerlist:
    df = pd.read_csv('Companies/{}.csv'.format(ticker), index_col=0)
    days = 1
    df['pred_cp'] = df['Adj Close'].shift(-days)
    df['High'] = df['High'].shift(-days)
    df['Low'] = df['Low'].shift(-days)
    df['Open'] = df['Open'].shift(-days)
    df['Close'] = df['Close'].shift(-days)
    df.to_csv("Companies/{}.csv".format(ticker), header=True)
    df.dropna(inplace=True)
    
    X = np.array(df[['Open', 'High', 'Low', 'Close', '50_mVA', '200_mvA']])
    Y = np.array(df['pred_cp'])

    lab_enc = preprocessing.LabelEncoder()
    encoded = lab_enc.fit_transform(Y)
    knn = KNeighborsClassifier(n_neighbors=2)

    sfs = SFS(knn, k_features=5, forward=True, floating=False, verbose=2, scoring='accuracy', cv=0)

    feature_names = ('Open', 'High', 'Low', 'Close', '50_mVA', '200mVA')
    sfs = sfs.fit(X, encoded, custom_feature_names=feature_names)
    kfeature.append(sfs.k_feature_idx_)
    kscore.append(sfs.k_score_)
    
    X = preprocessing.scale(X)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=1)
    clf = LinearRegression()
    clf.fit(X_train, Y_train)
    confidence = clf.score(X_test, Y_test)
    confidence1.append(confidence)

    X = X[:-days]
    X_new = X[-days:]
    prediction = clf.predict(X_new)
    
    predictive_values.append(float(np.ma.round(prediction, decimals=3)))

In [ ]:
merged_dataframe = pd.DataFrame(columns=['Date', 
                                         'Companies', 
                                         'Open', 
                                         'High', 
                                         'Low',
                                         'Close',
                                         '50d Moving AVG',
                                         '200d Moving AVG',
                                         'Predictive Close',
                                         'Confidence',
                                         'k_feature',
                                         'k_score'])

merged_dataframe['Date'] = Datelist
merged_dataframe['Companies'] = new_tickerlist
merged_dataframe['Open'] = open2
merged_dataframe['High'] = high1
merged_dataframe['Low'] = low1
merged_dataframe['Close'] = close1
merged_dataframe['50d Moving AVG'] = mva50all
merged_dataframe['200d Moving AVG'] = mva200all
merged_dataframe['Predictive Close'] = predictive_values
merged_dataframe['Confidence'] = confidence1
merged_dataframe['k_feature'] = kfeature
merged_dataframe['k_score'] = kscore

merged_dataframe

In [ ]:
shutil.rmtree('Companies')
shutil.rmtree('Companies_data')